In [2]:
from langchain.chat_models import ChatOpenAI
from langchain.callbacks import get_openai_callback

chat = ChatOpenAI(temperature=0.1)

with get_openai_callback() as usage:
    a = chat.predict("What is the recipe for soju")
    b = chat.predict("What is the recipe for bread")
    print(a, b, "\n", usage)

Tokens Used: 298
	Prompt Tokens: 14
	Completion Tokens: 284
Successful Requests: 1
Total Cost (USD): $0.0005889999999999999
